Sebastien Chassot <sebastien.chassot@etu.hesge.ch><br>
_(GnuPG 0x3F432F8E)_ <br>
HES-SO hepia ITI <br>
[http://hepia.hesge.ch](http://hepia.hesge.ch)<br>

* * * 
***

# Test



In [6]:
%%bash
netstat -ant|grep 8888

tcp        0      0 127.0.0.1:8888          0.0.0.0:*               LISTEN     
tcp6       0      0 ::1:8888                :::*                    LISTEN     
tcp6       0      0 ::1:51408               ::1:8888                ESTABLISHED
tcp6       0   1177 ::1:8888                ::1:51414               ESTABLISHED
tcp6       0      0 ::1:8888                ::1:51412               ESTABLISHED
tcp6     563      0 ::1:51414               ::1:8888                ESTABLISHED
tcp6       0      0 ::1:51411               ::1:8888                TIME_WAIT  
tcp6       0      0 ::1:51405               ::1:8888                ESTABLISHED
tcp6       0      0 ::1:51404               ::1:8888                TIME_WAIT  
tcp6       0      0 ::1:8888                ::1:51406               ESTABLISHED
tcp6       0      0 ::1:51410               ::1:8888                ESTABLISHED
tcp6       0      0 ::1:8888                ::1:51408               ESTABLISHED
tcp6       0      0 ::1:8888            

## Creation du system de fichier minix ##

On créé un système de fichier que l'on format.<br>

On monte le système de fichier afin d'écrire une serie de fichier


In [12]:
%%bash

truncate -s 20M ./filesystems/new.minixfs.img
mkfs.minix -1 ./filesystems/new.minixfs.img
#mount -o loop -t minix ./filesystems/new.minixfs.img /mnt
if mount|grep mini;
    then
    for i in {1..50}; do touch /mnt/file_$i;done
    
    echo "Un contenu ...avec" > /mnt/file_14
    echo -e "\n\nStat sur un fichier : \n"
    stat /mnt/file_14
    echo
    #ls -al /mnt
else
    echo "not mounted"
fi

6848 inodes
20480 blocks
Firstdatazone=220 (220)
Zonesize=1024
Maxsize=268966912

/home/seba/documents/hepia/tp-minixfs/filesystems/new.minixfs.img on /mnt type minix (rw,relatime)


Stat sur un fichier : 

  File: ‘/mnt/file_14’
  Size: 19        	Blocks: 2          IO Block: 1024   regular file
Device: 702h/1794d	Inode: 15          Links: 1
Access: (0644/-rw-r--r--)  Uid: ( 1000/    seba)   Gid: ( 1000/    seba)
Access: 2015-06-07 23:38:38.000000000 +0200
Modify: 2015-06-07 23:38:38.000000000 +0200
Change: 2015-06-07 23:38:38.000000000 +0200
 Birth: -



## Verification de l'état du disque ##

on fait un hexdump de certain block


<br>

In [8]:
%%python2
from constantes import *
import sys
import constantes
import bloc_device as blk
import hexdump
import os
import sys

testfile="filesystems/new.minixfs.img"

disk = blk.bloc_device(BLOCK_SIZE, testfile)

hexdump.hexdump(disk.read_bloc(221))
del disk

00000000: 20 00 66 69 6C 65 5F 33  31 00 00 00 00 00 00 00   .file_31.......
00000010: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000020: 21 00 66 69 6C 65 5F 33  32 00 00 00 00 00 00 00  !.file_32.......
00000030: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000040: 22 00 66 69 6C 65 5F 33  33 00 00 00 00 00 00 00  ".file_33.......
00000050: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000060: 23 00 66 69 6C 65 5F 33  34 00 00 00 00 00 00 00  #.file_34.......
00000070: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000080: 24 00 66 69 6C 65 5F 33  35 00 00 00 00 00 00 00  $.file_35.......
00000090: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000000A0: 25 00 66 69 6C 65 5F 33  36 00 00 00 00 00 00 00  %.file_36.......
000000B0: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
000000C0: 26 00 66 69 6C 65 5F 33  37 00 00 00 00 00 00 00  &.file_37.......

INFO:file system opened successfully
INFO:file descriptor closed


## Test d'effacement d'une entrée

On efface une entrée puis on remarque que l'inode de l'entrée vaut maintenant 0x00


### On sature le disque avec balloc appelé en boucle ###


<br>


In [13]:
%%python2
testfile="filesystems/new.minixfs.img"
import minixfs as mx
import hexdump
minixfs=mx.minix_file_system(testfile)

minixfs.del_entry(minixfs.inodes_list[1], "file_38")
hexdump.hexdump(minixfs.disk.read_bloc(221)[192:256])
#print(minixfs.lookup_entry(minixfs.inodes_list[1], 'file_1'))
#for i in range(100000):
#    minixfs.balloc()
    
del minixfs

00000000: 26 00 66 69 6C 65 5F 33  37 00 00 00 00 00 00 00  &.file_37.......
00000010: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................
00000020: 00 00 66 69 6C 65 5F 33  38 00 00 00 00 00 00 00  ..file_38.......
00000030: 00 00 00 00 00 00 00 00  00 00 00 00 00 00 00 00  ................


INFO:file system opened successfully
INFO:file descriptor closed
INFO:Close minix filesystem


## Test allocation d'un block doublement indirect

On utilise ialloc_bloc() pour créer et écrire dans un block doublement indirect


<br>


In [16]:
%%python2
testfile="filesystems/new.minixfs.img"
import minixfs as mx
import hexdump

filename="file_128"
minixfs=mx.minix_file_system(testfile)


minixfs.add_entry(minixfs.inodes_list[1], filename, minixfs.balloc())
inode = minixfs.lookup_entry(minixfs.inodes_list[1], filename)
print(inode)
minixfs.ialloc_bloc(minixfs.inodes_list[inode], 8)

hexdump.hexdump(minixfs.disk.read_bloc(minixfs.bmap(minixfs.inodes_list[inode]), 5000))
#print(minixfs.lookup_entry(minixfs.inodes_list[1], 'file_1'))

del minixfs

224


INFO:file system opened successfully
ERROR:Error Write_block: Unable to write requested block to disk
Traceback (most recent call last):
  File "<stdin>", line 12, in <module>
  File "minixfs.py", line 255, in ialloc_bloc
    self.disk.write_bloc(bytearray("".ljust(1024, '\x00')), inode.i_indir_zone)
  File "bloc_device.py", line 102, in write_bloc
    raise BlocDeviceException("Error Write_block: Unable to write requested block to disk")
bloc_device.BlocDeviceException: Error Write_block: Unable to write requested block to disk
INFO:file descriptor closed
INFO:Close minix filesystem


## Teste bmap ##

#####TODO #####
Verifier ce que renvoi bmap quand block inexistant. Est-ce que la taille change et comment faire?

bmap doit peut-être tester si bmap dépasse la taille spécifiée dans l'inode?

In [51]:
%%bash

tcpdump -# -t -n -c 1000 -xx -r docs/tcpdump_hepia.cap

    1  IP 192.168.0.18.34390 > 129.194.184.113.1234: Flags [S], seq 2117175925, win 29200, options [mss 1460,sackOK,TS val 6387904 ecr 0,nop,wscale 7], length 0
	0x0000:  fc94 e3a8 4a80 f46d 0494 18d2 0800 4500
	0x0010:  003c f2b5 4000 4006 4d18 c0a8 0012 81c2
	0x0020:  b871 8656 04d2 7e31 8a75 0000 0000 a002
	0x0030:  7210 ce10 0000 0204 05b4 0402 080a 0061
	0x0040:  78c0 0000 0000 0103 0307
    2  IP 129.194.184.113.1234 > 192.168.0.18.34390: Flags [S.], seq 92836250, ack 2117175926, win 14480, options [mss 1380,sackOK,TS val 219509930 ecr 6387904,nop,wscale 7], length 0
	0x0000:  f46d 0494 18d2 fc94 e3a8 4a80 0800 4500
	0x0010:  003c 0000 4000 3806 47ce 81c2 b871 c0a8
	0x0020:  0012 04d2 8656 0588 919a 7e31 8a76 a012
	0x0030:  3890 eeed 0000 0204 0564 0402 080a 0d15
	0x0040:  74aa 0061 78c0 0103 0307
    3  IP 192.168.0.18.34390 > 129.194.184.113.1234: Flags [.], ack 1, win 229, options [nop,nop,TS val 6387908 ecr 219509930], length 0
	0x0000:  fc94 e3a8 4a80 f46d 0494 18d2 0800 450

reading from file docs/tcpdump_hepia.cap, link-type EN10MB (Ethernet)


In [27]:
%%python2
import scapy
from scapy.all import *
from scapy.utils import rdpcap

pkts=rdpcap("./docs/tcpdump_hepia.cap")

for i in [1, 2, 3, 5, 226, 227]:
    print("\n\nPacket : "+ i.__str__()+ "\n")
    hexdump(pkts[i])
    print("\n")
    pkts[i].show2()




Packet : 1

0000   F4 6D 04 94 18 D2 FC 94  E3 A8 4A 80 08 00 45 00   .m........J...E.
0010   00 3C 00 00 40 00 38 06  47 CE 81 C2 B8 71 C0 A8   .<..@.8.G....q..
0020   00 12 04 D2 86 56 05 88  91 9A 7E 31 8A 76 A0 12   .....V....~1.v..
0030   38 90 EE ED 00 00 02 04  05 64 04 02 08 0A 0D 15   8........d......
0040   74 AA 00 61 78 C0 01 03  03 07                     t..ax.....


###[ Ethernet ]###
  dst       = f4:6d:04:94:18:d2
  src       = fc:94:e3:a8:4a:80
  type      = 0x800
###[ IP ]###
     version   = 4L
     ihl       = 5L
     tos       = 0x0
     len       = 60
     id        = 0
     flags     = DF
     frag      = 0L
     ttl       = 56
     proto     = tcp
     chksum    = 0x47ce
     src       = 129.194.184.113
     dst       = 192.168.0.18
     \options   \
###[ TCP ]###
        sport     = search_agent
        dport     = 34390
        seq       = 92836250
        ack       = 2117175926
        dataofs   = 10L
        reserved  = 0L
        flags     = SA
        wi

In [47]:
!whois 129.194.184.113 |grep -3 inetnum


% No abuse contact registered for 129.194.0.0 - 129.194.255.255

inetnum:        129.194.0.0 - 129.194.255.255
netname:        UNIGE-CENTER
descr:          University of Geneva-Ctr
descr:          Geneva, Switzerland


In [6]:
%%python2
import minixfs as mx
help(mx)

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "minixfs.py", line 14, in <module>
    from minix_inode import *
  File "minix_inode.py", line 13, in <module>
    from constantes import *
  File "constantes.py", line 18, in <module>
    import hexdump
ImportError: No module named hexdump


In [56]:
%%bash
strace ./server 1234 ./filesystems/remote_minixfs_lab1.img

Process is terminated.
